# 3D CNN Model for MRI Image Processing
This notebook demonstrates how to use a 3D CNN model for processing MRI images to determine the presence of Alzheimer's Disease (AD).

## Data Loading and Preprocessing


In [4]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Load the CSV file
data_df = pd.read_csv('structured_data_mac.csv')

# Initialize X (images) and y (labels) lists
X = []
y = []

# Load and preprocess images
for _, row in data_df.iterrows():
    image_path = row['Image Path']
    image = nib.load(image_path).get_fdata()

    # Normalize image data
    image = image / np.max(image)

    X.append(image)
    y.append(row['Group'])  # Use the 'Group' column as labels

# Convert category labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert integer category labels to one-hot encoding
y_categorical = to_categorical(y_encoded)

# Convert X to a numpy array
X = np.array(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42)

## Defining the 3D CNN Model

Below is a basic structure of a 3D CNN model.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

def build_model():
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(96, 96, 96, 1)), 
        MaxPooling3D(pool_size=(2, 2, 2)),
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


## Training the Model
We will train the model on the training set and use a portion of the data as a validation set.


In [6]:
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.2)


: 

## Evaluating the Model
Finally, we will evaluate the performance of the model on the test set.


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")
